In [ ]:
!pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.3 MB/s eta 0:00:00


In [ ]:
from rapidfuzz import process, fuzz
import pandas as pd
import datetime

def generate_timestamp():
    return datetime.datetime.now().strftime("%Y%m%d%H%M%S")

def custom_mapping(import_df, template_cols):
    matched_columns = {}
    import_columns = import_df.columns.tolist()

    # Attempt to directly match or fuzzily match "Years Worked" to "Tenure (Years)"
    tenure_aliases = ["Tenure (Years)", "Years Worked", "Years_Worked"]  # Possible variations
    tenure_col_found = None

    for alias in tenure_aliases:
        if alias in import_columns:
            tenure_col_found = alias
            break
        else:
            # Fuzzy match for tenure-related column
            best_match, score, _ = process.extractOne(alias, import_columns, scorer=fuzz.WRatio)
            if score > 80:
                tenure_col_found = best_match
                break

    if tenure_col_found:
        matched_columns["Tenure (Years)"] = tenure_col_found

    # Additional direct and fuzzy mappings for other columns
    for t_col in template_cols:
        if t_col in ["Tenure (Years)"]:  # Skip already handled columns
            continue
        if t_col in import_columns:
            matched_columns[t_col] = t_col
        else:
            best_match, score, _ = process.extractOne(t_col, import_columns, scorer=fuzz.WRatio)
            if score > 80:
                matched_columns[t_col] = best_match

    # Handling "Name" column split
    if 'Name' in import_columns and ('First Name' not in import_columns or 'Last Name' not in import_columns):
        import_df[['First Name', 'Last Name']] = import_df['Name'].str.split(' ', 1, expand=True)
        matched_columns['First Name'] = 'First Name'
        matched_columns['Last Name'] = 'Last Name'

    return matched_columns

# Assuming the file paths are set to your CSV files
template_df = pd.read_csv('Example_Template_File.csv')
import_df = pd.read_csv('Import_File.csv')
required_columns = template_df.columns.tolist()

# Apply custom mapping rules
matched_columns = custom_mapping(import_df, required_columns)

# Prepare the output DataFrame
output_df = pd.DataFrame()

# Fill the output DataFrame based on matched columns
for t_col in required_columns:
    matched_col = matched_columns.get(t_col)
    if matched_col:
        output_df[t_col] = import_df[matched_col]
    else:
        output_df[t_col] = pd.NA  # Fill missing template columns with NaN

# Generate and save the output file
output_file_name = f"output_{generate_timestamp()}.csv"
output_df.to_csv(output_file_name, index=False)

print(f"Output file created: {output_file_name}")


Output file created: output_20240305202640.csv


<ipython-input-3-2b360668d403>:43: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  import_df[['First Name', 'Last Name']] = import_df['Name'].str.split(' ', 1, expand=True)
